# Optimization

Typically, transportation policy planning models will be used to
try to find policies that provide the "best" outcomes.  In a traditional
analytical environment, that typically means using models to find optimal outcomes
for performance measures, possibly subject to some assumptions about the
future values of various possible exogenous uncertainties that may 
impact the models.

## Optimization Tools

Transportation models as used in the TMIP-EMAT frawework are 
generally characterized by two important features: they are 
subject to significant exogenous uncertainties about the future
state of the world, and they include numerous performance measures
for which decision makers would like to see good outcomes. Therefore,
optimization tools applied to these models should be 
flexible to consider *multiple objectives*,
as well as be *robust* against uncertainty.

### Multi-Objective Optimization

With exploratory modeling, optimization is also often undertaken as a 
[multi-objective optimization](https://en.wikipedia.org/wiki/Multi-objective_optimization) exercise, where multiple
and possibly conflicting performance measures need to be addressed simultaneously. 
A road capacity expansion project is a good example of a multi-objective optimization 
problem. In such a situation, we want to expand the capacity of a roadway, both minimizing
the costs and maximizing the travel time benefits.  A smaller expansion project will cost less
but also provide lesser benefits.  Funding it with variable rate debt might 
decrease expected future costs but doing so entails more risk than fixed-rate debt.

One approach to managing a multi-objective optimization problem is to distill it 
into a single objective problem, by assigning relative weights to the various 
objectives.  For a variety of reasons, this can be difficult to accomplish in public policy environments that are common in transportation planning. 
Multiple stakeholders may have different
priorities and may not be able to agree on a relative weighting structure.  
Certain small improvements in a performance measure may be valued very differently 
if they tip the measure over a regulated threshold (e.g. to attain a particular 
mandated level of emissions or air quality).

Instead of trying to simplify a multi-objective into a simple-objective one,
an alternate approach is to preserve the multi-objective nature of the problem
and find a set or spectrum of different solutions, each of which solves the problem
at a different weighting of the various objectives.  Decision makers can then
review the various different solutions, and make judgements about the various
trade-offs implicit in choosing one path over another.

Within a set of solutions for this kind of problem, each individual solution is 
"[Pareto optimal](https://en.wikipedia.org/wiki/Pareto_efficiency)", such that
no individual objective can be improved without degrading at least one other
objective by some amount.  Thus, each of these solutions might be the "best"
policy to adopt, and exactly which is the best is left as a subjective judgement
to decision makers, instead of being a concretely objective evaluation based 
on mathematics alone.

### Robust Optimization

Robust optimization is a variant of the more traditional optimization
problem, where we will try to find policies that yield *good* outcomes
across a range of possible futures, instead of trying to find a policy
that delivers the *best* outcome for a particular future.

To conceptualize this, let us consider a decision where there are four
possible policies to choose among, a single exogenous uncertainty that
will impact the future, and a single performance measure that we would
like to mimimize.  We have a model that can forecast the performance 
measure, conditional on the chosen policy and the future value of the
exogenous uncertainty, and which gives us a forecasts as shown below.

In [1]:
import numpy
from matplotlib import pyplot as plt
x = numpy.linspace(0,1)
y1 = x**3 + numpy.cos((x-3)*23)*.01
y2 = x**3 + .1 + numpy.sin((x-3)*23)*.01
y3 = 1.3*(1-x)**17 + numpy.cos((x-3)*23)*.01 + .1
y4 = numpy.sin((x-3)*23)*.01+0.16 + .1*(x-0.5)

linestyles = [
    dict(ls='-', c='blue'),
    dict(ls=':', c='orange'),
    dict(ls='-.', c='green'),
    dict(ls='--', c='red'),
]

fig, ax = plt.subplots(1,1)
ax.plot(x, y1, **linestyles[0], label="Policy 1")
ax.plot(x, y2, **linestyles[1], label="Policy 2")
ax.plot(x, y3, **linestyles[2], label="Policy 3")
ax.plot(x, y4, **linestyles[3], label="Policy 4")
ax.set_ylabel("Performance Measure\n← Lower is Better ←")
ax.set_xlabel("Exogenous Uncertainty")
ax.legend()
ax.set_title("Example Simple Forecasting Experiment")

plt.savefig("robust_example.png");

In a naive optimization approach, if we want to minimize the performance
measure, we can do so by selecting Policy 1 and setting the exogenous
uncertainty to 0.1.  Of course, in application we are able to select
Policy 1, but we are unable to actually control the exogenous uncertainty
(hence, "exogenous") and we may very well end up with a very bad result
on the right side of the figure.

We can see from the figure that, depending on the ultimate value for 
the exogenous uncertainty, either Policy 1 or Policy 3 might yield the
best possible value of the performace measure.  However, both of these
policies come with substantial risks as well -- in each Policy there are
some futures where the results are optimal, but there are also some 
futures where the results are exceptionally poor.

In contrast with these optimal policies, Policy 4 may be considered a 
"robust" solution.  Although there is no value of the exogenous 
uncertainty where Policy 4 yields the best possible outcome, there is
also no future where Policy 4 yields a very poor outcome.  Instead, across
all futures it is always generating a "pretty good" outcome.

Different expectations for the future may lead to different policy 
choices.  If the decision maker feels that low values of the exogenous
uncertainty are much more likely than high values, Policy 1 might be
the best policy to choose.  If high values of the exogenous uncertainty
are expected, then Policy 3 might be the best choice.  If there is not
much agreement on the probable future values of the exogenous uncertainty,
or if decision makers want to adopt a risk-averse stance, then Policy 4
might be the best choice.  

The remaining policy shown in the figure, Policy 2, is the lone holdout 
in this example -- there is no set of expectations about the future, or
attitudes toward risk, that can make this policy the best choice.  This
is because, no matter what the future value of the exogenous uncertainty
may be, the performance measure has a better outcome from Policy 1 than
from Policy 2.  In this circumstance, we can say that Policy 2 is 
"dominated" and should never be chosen by decision makers.

#### Robustness Functions

To perform robust optimization in EMAT, we need a core model (or meta-model)
with defined input and output parameters, as well a set of functions called
"robustness measures" that define what a "robust" measure represents.  
As noted above, different expectations about future states of the world
can lead to different rankings of policy options.  In addition, different
attitudes towards risk can result in different robustness measures that are
derived from the same underlying modeled performance measures.  

For example, consider the *Example Simple Forecasting Experiment* shown above.
In this example, we could compute a robustness measure for each policy where
we calculate the maximum value of the performance measure across any possible
value of the exogenous uncertainty.  This value is shown in the first column
of robustness measures in the figure below, and under this measure Policy 4 is
far and away the best choice. 

In [ ]:
fig, ax = plt.subplots(
    1,6, sharey=True, 
    gridspec_kw=dict(width_ratios=[7,1,1,1,1,1], wspace=0.05,), 
    figsize=[8, 4.],
)


ax[0].plot(x, y1, **linestyles[0], label="Policy 1")
ax[0].plot(x, y2, **linestyles[1], label="Policy 2")
ax[0].plot(x, y3, **linestyles[2], label="Policy 3")
ax[0].plot(x, y4, **linestyles[3], label="Policy 4")
ax[1].plot([0,1], [y1.max()]*2, **linestyles[0], lw=2)
ax[1].plot([0,1], [y2.max()]*2, **linestyles[1], lw=2)
ax[1].plot([0,1],[y3.max()]*2, **linestyles[2],  lw=2)
ax[1].plot([0,1], [y4.max()]*2, **linestyles[3], lw=2)

ax[2].plot([0,1], [y1.mean()]*2, **linestyles[0], lw=2)
ax[2].plot([0,1], [y2.mean()]*2, **linestyles[1], lw=2)
ax[2].plot([0,1],[y3.mean()]*2, **linestyles[2],  lw=2)
ax[2].plot([0,1], [y4.mean()]*2, **linestyles[3], lw=2)

ax[3].plot([0,1], [numpy.median(y1)]*2, **linestyles[0], lw=2)
ax[3].plot([0,1], [numpy.median(y2)]*2, **linestyles[1], lw=2)
ax[3].plot([0,1], [numpy.median(y3)]*2, **linestyles[2],  lw=2)
ax[3].plot([0,1], [numpy.median(y4)]*2, **linestyles[3], lw=2)

ax[4].plot([0,1], [numpy.percentile(y1, 90)]*2, **linestyles[0], lw=2)
ax[4].plot([0,1], [numpy.percentile(y2, 90)]*2, **linestyles[1], lw=2)
ax[4].plot([0,1], [numpy.percentile(y3, 90)]*2, **linestyles[2],  lw=2)
ax[4].plot([0,1], [numpy.percentile(y4, 90)]*2, **linestyles[3], lw=2)

ax[5].plot([0,1], [y1.min()]*2, **linestyles[0], lw=2)
ax[5].plot([0,1], [y2.min()]*2, **linestyles[1], lw=2)
ax[5].plot([0,1], [y3.min()]*2, **linestyles[2], lw=2)
ax[5].plot([0,1], [y4.min()]*2, **linestyles[3], lw=2)


ax[0].set_ylabel("Performance Measure\n← Lower is Better ←")
ax[0].set_xlabel("Exogenous Uncertainty")
ax[0].legend()
ax[0].set_title("Example Simple Forecasting Experiment")
ax[3].set_title("Robustness Measures")
ax[1].set_xlabel("Max\nPM")
ax[2].set_xlabel("Mean\nPM")
ax[3].set_xlabel("Median\nPM")
ax[4].set_xlabel("90%ile\nPM")
ax[5].set_xlabel("Min\nPM")
ax[-1].yaxis.set_label_position("right")
for a in [1,2,3,4,5]:
    ax[a].set_xticks([])
    ax[a].set_yticks([])
plt.savefig("robust_measures.png");


The "maximum performance measure result" robustness function is a very
risk averse approach, as no consideration is given to the shape or distribution
of performance measure values other than the maximum.  Consider these same
policies shown if Policy 4 is not available. In this case, the next best 
policy under this robustness function is Policy 1, as it has the next lowest
maximum value.  However, when we look at
a comparison between Policies 1 and 3 in aggregate, we might easily conclude
that Policy 3 is a better choice overall: it is better than Policy 1 on average,
as judged by the mean, median, and 90th percentile measures.  The only reason 
Policy 3 appears worse than Policy 1 on the initial robustness function is that 
it has an especially poor outcome at one extreme end of the uncertainty 
distribution.  Depending on our attitude towards risk on this performance measure,
we may want to consider using some of these alternative robustness functions.
An additional consideration here is that the various robustness measures
in this example case are all unweighted measures: they implicitly assume a 
uniform probability distribution for the entire range of possible values for
the exogenous uncertainty.  If we are able to develop a probability distribution
on our expectations for the future values of the exogenous uncertainties,
we can use that probability distribution to weight the robustness functions
appropriately, creating more meaningful values, particularly for the non-extreme
value robustness functions (i.e., everything except the min and max).

## Mechanics of Using Optimization

### Policy Optimization: Search over Levers

The simplest optimization tool available for TMIP-EMAT users is 
a *search over policy levers*, which represents multi-objective
optimization, manipulating policy lever values to find a Pareto
optimal set of solutions, holding the exogenous uncertainties
fixed at a particular value for each uncertainty (typically at 
the default values).  This is often a useful first step
in exploratory analysis, even if your ultimate goal is to 
eventually undertake a robust optimization analysis.  This less
complex optimization can give insights into tradeoffs between
performance measures and reasonable combinations of policy levers.

To demonstrate a search over levers, we'll use the Road Test 
example model.

In [2]:
import emat.examples
scope, db, model = emat.examples.road_test()

The scope defined for a model in TMIP-EMAT will already provide
information about the preferred directionality of performance
measures (i.e., 'maximize' when larger values are better,
'minimize' when smaller values are better, or 'info' when we
do not have a preference for bigger or smaller values, but we 
just want to be tracking the measure).  We can see these
preferences for any particular performance measure by inspecting
the scope definition file, or by using the `info` method of
`emat.Measure` instances.

In [3]:
scope['net_benefits'].info()

net_benefits:
  kind: maximize


To conduct an optimization search over levers, we'll use the 
`optimize` method of the TMIP-EMAT model class, setting the
`search_over` argument to `'levers'`.  In this example, we will
set the number of function evaluations (`nfe`) to 10,000, although
for other models you may need more or fewer to achieve a good, 
well converged result.  In a Jupyter notebook environment, 
we can monitor convergence visually in real time in the figures
that will appear automatically.

In [4]:
result = model.optimize(
    nfe=10_000, 
    searchover='levers', 
    check_extremes=1,
    cache_file='./optimization_cache/road_test_search_over_levers.gz',
)

ConvergenceMetrics(children=(FigureWidget({
    'data': [{'fill': 'tonexty',
              'line': {'shape': '…

The `optimize` method returns an `OptimizationResult` object, 
which contains the resulting solutions, as well as some information
about how they were derived.  We can review the raw values of the
solutions as a pandas DataFrame, or see the scenario values used 
to generate these solutions.

In [5]:
result.result.head()

,expand_capacity,amortization_period,debt_type,interest_rate_lock,build_travel_time,time_savings,value_of_time_savings,net_benefits,cost_of_capacity_expansion
0,43.450483,50,Paygo,True,62.125370,6.874630,6.874630,-103.979417,110.854047
1,15.332477,50,Paygo,True,65.086699,3.913301,3.913301,-35.204034,39.117335
2,92.259130,50,Paygo,False,60.658711,8.341289,8.341289,-227.036946,235.378235
3,3.931966,50,Paygo,True,67.713402,1.286598,1.286598,-8.744919,10.031518
4,63.297905,50,Paygo,False,61.265669,7.734331,7.734331,-153.755912,161.490243


In [6]:
result.scenario

Scenario({'alpha': 0.15, 'beta': 4.0, 'input_flow': 100, 'value_of_time': 0.01, 'unit_cost_expansion': 100, 'interest_rate': 0.03, 'yield_curve': 0.01})

We can visualize the set of solutions using a
[parallel coordinates](https://en.wikipedia.org/wiki/Parallel_coordinates) 
plot.  This figure is composed of a number of vertical axes, one for each 
column of data in the results DataFrame.  Each row of the DataFrame is
represented by a chord that connects across each of the vertical axes.

By default, the axes representing performace measures to be minimized are
inverted in the parallel coordinates, such that moving up along any 
performance measure axis results in a "better" outcome for that performance
measure.

In the figure below, we can quickly see that all the all of the Pareto
optimal policy solutions for our reference scenario share an amortization
period of 50 years and a debt type of 'Paygo'.  By contrast, the set of 
solutions include multiple different values for the expand capacity lever,
ranging from 0 to 100.  These different values offer possible tradeoffs 
among the performance measures: lower levels of capacity expansion (shown
in yellow) will maximize net benefits and minimize the cost of the project,
but they will also fail to provide much travel time savings.  Conversely,
larger levels of capacity expansion will provide better travel time savings, 
but will not perform as well on costs.  It is left up the the analysts and
decision makers to judge what tradeoffs to make between these conflicting
goals.

In [7]:
result.par_coords()

ParCoordsViewer(children=(FigureWidget({
    'data': [{'dimensions': [{'label': '(L) expand_capacity',
       …

### Worst Case Discovery: Search over Uncertainties

We can apply the same multiobjective optimization tool in reverse to 
study the worst case outcomes from any particular set of policy lever
settings.  To do so, we switch out the `searchover` argument from
`'levers'` to `'uncertainties'`, and set `reverse_targets` to True,
which will tell the optimization engine to search for the worst outcomes
instead of the best ones.  We'll often also want to override the reference
policy values with selected particular values, although it's possible
to omit this reference argument to search for the worst case scenarios
under the default policy lever settings.

In [8]:
worst = model.optimize(
    nfe=10_000, 
    searchover='uncertainties', 
    reverse_targets = True,
    check_extremes=1,
    cache_file='./optimization_cache/road_test_search_over_uncs.gz',
    reference={
        'expand_capacity': 100.0, 
        'amortization_period': 50, 
        'debt_type': 'PayGo', 
        'interest_rate_lock': False,
    }
)

ConvergenceMetrics(children=(FigureWidget({
    'data': [{'fill': 'tonexty',
              'line': {'shape': '…

In [9]:
worst.par_coords()

ParCoordsViewer(children=(FigureWidget({
    'data': [{'dimensions': [{'label': '(X) alpha',
                 …

### Robust Optimization

As dicsussed above, implementing robust optimization requires the
definition of relevant robustness functions.  Because the functional
form of these functions can be so many different things depending on
the particular application, TMIP-EMAT does not implement a mechanism
to generate them automatically.  Instead, it is left to the analyst to
develop a set of robustness functions that are appropriate for each
application.

Each robustness function represents an aggregate performance measure,
calculated based on the results from a group of individual runs of the
underlying core model (or meta-model).  These runs are conducted with
a common set of policy lever settings, and a variety of different 
exogenous uncertainty scenarios.  This allows us to create aggregate 
measures that encapsulate not only a point value but also a statistical
measurement of the distribution of the actual performance measures
output from the model.


In [ ]:
expected_net_benefit = Measure(
    name='Expected Net Benefit',
    kind=Measure.MAXIMIZE,
    variable_name='net_benefits',
    function=numpy.mean,
)


In [ ]:
expected_build_travel_time = Measure(
        name='Expected Build Travel Time',
        kind=Measure.INFO, 
        variable_name='build_travel_time', 
        function=numpy.mean,
)

In [ ]:
robustness_functions = [
    Measure(
        'Expected Build Travel Time',
        kind = Measure.INFO, 
        variable_name = 'build_travel_time', 
        function = numpy.mean,
    ),
    Measure(
        'Expected Time Savings',
        kind = Measure.INFO, 
        variable_name = 'time_savings', 
        function = numpy.mean,
    ),
    Measure(
        'Expected Net Benefit',
        kind = Measure.MAXIMIZE,
        variable_name = 'net_benefits',
        function = numpy.mean, min = 0, max = 1000,
    ),
    Measure(
        '95%ile Capacity Expansion Cost',
        kind = Measure.MINIMIZE,
        variable_name = 'cost_of_capacity_expansion', 
        function = functools.partial(numpy.percentile, q = 95), 
    ),
    Measure(
        '99%ile Value of Time Saving',
        kind = Measure.MAXIMIZE, 
        variable_name = 'value_of_time_savings',
        function = functools.partial(numpy.percentile, q = 99),
    )
]

We may also want to use some constraints to define our desired solution space for optimization. It is also a good idea to track the progress of optimization using appropriate convergence metrics.

In this notebook, we will use EMAT to demonstrate how we can implement robust optimization to obtain optimal decisions. We will consider an example of road capacity expansion project to find out feasible decision alternatives regarding the investment for capacity expansion on a single network link. We will use BPR function as the volume-delay function to calculate the average travel time for various scenarios. The parameters in the core model can be constants, uncertainties and/or policy levers. Some examples of each type of model parameters are listed below:
* Constant: Free flow travel time, initial capacity, etc.
* Uncertainty: BPR alpha and beta parameter, average flow on the link, per unit cost of capacity expansion, interest rate, etc.
* Policy Lever: Expected capacity expansion, amortization period, type of financing, etc. 

The outputs from this model can be referred to as outcome measures such as average travel time and travel time savings after capacity expansion, the net benefit of the project due to expansion and annual payment amount to finance the expansion. We will use robust optimization, in this case, to find out a set of solutions depending on these measures that are used as the robustness metrics in the optimization process. For example, we may expect to see what our decisions should be regarding the unit cost of capacity expansion, interest rate, amortization period, and type of financing in order to maximize the value of time savings due to expansion. In other words, we want to obtain solutions that interpret the relations between selected performance measures and decision parameters which include both policy levers and uncertainties. Likewise, it is also possible to understand the relationship between different outcome measures and their relative impact on each other. For instance, we can examine what the expected value of time savings would be if we plan to minimize the unit cost of capacity expansion.

#### Getting Started

We will start with importing a bunch of python packages along with `emat` that we need for robust optimization. We also need a scope file to create a model object where uncertainties, policy levers and performance measures for core/meta-models are defined using a recommended format. We use `emat.Scope()` to create a `Scope` object. Then, we create a database and store the `Scope` object using `.store_scope(emat.SQLiteDB())`. We can view the names of the scopes stored in database using `.read_scope_names()` method. We may turn on the logger to record logs of all processes involved.

In [ ]:
# Import packages

import emat, ema_workbench, os, numpy, pandas, functools
from emat.util.xmle import Show
import matplotlib.pyplot as plt
logger = emat.util.loggers.log_to_stderr(20, True)

In [ ]:
# Create scope object and store in database

road_scope = emat.Scope(emat.package_file('model','tests','road_test.yaml'))
emat_db = emat.SQLiteDB()
road_scope.store_scope(emat_db)
emat_db.read_scope_names()

#### Import core model

As mentioned earlier, we require a core model to perform robust optimization in `emat`. When we have the scope stored in our database, we import this core model to define a model object using `PythonCoreModel()` class. In this example, the core model is constructed as a class and referred to as `Road_Capacity_Investment`. We also need to import an evaluator for running the optimization process. For this demonstration, `SequentialEvaluator` from `ema_workbench` is used to simplify the process.

In [ ]:
# Import core model and evaluator 

from emat.model.core_python import PythonCoreModel
from emat.model.core_python import Road_Capacity_Investment
from ema_workbench import SequentialEvaluator

In [ ]:
# Define model object and evaluator

m = PythonCoreModel(Road_Capacity_Investment, scope = road_scope, db = emat_db)
eval_seq = SequentialEvaluator(m)

#### Define robustness functions

For decision analysis under data uncertainty, robustness metrics are used to quantify the robustness of policy levers over multiple scenarios using a single score. For robustness calculation, we need three types of specifications:
* A set of policy lever parameters for which we need to calculate robustness
* A set of performance metrics for these policy parameters
* A set of scenarios over which the performance of policy parameters will be evaluated

To learn more about how the robustness metrics are calculated, please refer to [McPhail et al. (2018)](https://agupubs.onlinelibrary.wiley.com/doi/full/10.1002/2017EF000649).

In case of robust optimization, we need to define these robustness metrics in the form of robustness functions. Each robustness function can be described as an outcome measure of the core model using a class in `emat` called `Measure()`. The first argument in `Measure()` is a `str` type name for the robustness metric. We can also set the preferred direction of measures (`INFO`, `MINIMIZE` and `MAXIMIZE`) using `kind` argument. The name of outcome measures obtained from the core model is specified by `variable_name`. Finally, a callable function can be defined in `function` to transform the outcome measure as robustness metric. `min` and `max` arguments specify the minimum and maximum value respectively that should be observed for any realization of uncertainties.

In this example, we define 5 robustness functions using `Measure()` method that essentially represent 5 performance metrics for the policy parameters. The first two functions are defined to indicate the expected level of performance using `INFO`. Then, we want to maximize the net benefit of this project and the value of time savings due to capacity expansion. In the meantime, we also want the unit cost for capacity expansion to be minimized. We can also consider percentile-based robustness metrics for minimization or maximization. If an outcome measure is not specified to be minimized or maximized in the optimization process, the respective `kind` argument is set to `INFO`by default.    

In [ ]:
# Import Measure and Constraint from emat to define robustness metrics and constraints

from emat import Measure, Constraint

In [ ]:
# Define robustness functions

robustness_functions = [Measure('Expected Build Travel Time', 
                                kind = Measure.INFO, 
                                variable_name = 'build_travel_time', 
                                function = numpy.mean,),
                        Measure('Expected Time Savings', 
                                kind = Measure.INFO, 
                                variable_name = 'time_savings', 
                                function = numpy.mean,),
                        Measure('Expected Net Benefit', 
                                kind = Measure.MAXIMIZE, 
                                variable_name = 'net_benefits', 
                                function = numpy.mean, min = 0, max = 1000),
                        Measure('95%ile Capacity Expansion Cost', 
                                kind = Measure.MINIMIZE, 
                                variable_name = 'cost_of_capacity_expansion', 
                                function = functools.partial(numpy.percentile, q = 95), min = 0, max = 800),
                        Measure('99%ile Value of Time Saving', 
                                kind = Measure.MAXIMIZE, 
                                variable_name = 'value_of_time_savings', 
                                function = functools.partial(numpy.percentile, q = 99), min = 0, max = 1000)]

#### Define constraints

The robust optimization process can be constrained to only include solutions that satisfy certain constraints. These constraints can be based on the policy lever parameters that are contained in the core model, the aggregate performance measures identified in the list of robustness functions, or some combination of levers and aggregate measures.  Importantly, the constraints *cannot* be imposed on the exogenous uncertainties, nor directly on the output measures from the core models (or the equivalent meta-model). This is because the robust version of the model aggregates a number of individual core model runs, and effectively hides these two components from the optimization engine.  

One way to work around this limitation, at least on the output measures, is to write robustness functions that transmit the relevant output measures through the aggregation process. For example, to constrain the robust search only to instances where a particular output measure is always positive, then write a robustness function that takes the *minimum* value of the targeted performance measure, and write a constraint that ensures that the minimum value is always positive.  This approach should be used with caution, however, as it may severely limit the search space.

In this example, we defined two constraints to consider solutions that are within the limited search space. Using the `Constraint()` method, we specify that the maximum interest rate must be less than 0.03 and minimum net benefit must be greater than 0. 

In [ ]:
# Define Constraints
constraint_1 = Constraint("Maximum Interest Rate", 
                          parameter_names = "interest_rate", 
                          function = Constraint.must_be_less_than(0.03),)

constraint_2 = Constraint("Minimum Net Benefit", 
                          outcome_names = "Expected Net Benefit", 
                          function = Constraint.must_be_greater_than(0),)

#### Specify convergence metrics

For any optimization problem, it is a good practice to keep track of the convergence progress using various metrics. `emat` comes with two types of convergence metrics - `HyperVolume` and `EpsilonProgress`. Both metrics are monotonous measures of convergence, so they are very useful to examine the convergence speed of optimization.

Hypervolume is an interesting convergence indicator in case of multi-objective optimization. It essentially defines the dominated multi-dimensional objective space within Pareto frontier. When the hyper-volume is calculated between a specified maximum and minimum value, this metric indicates the ratio of all dominated outcomes to all possible outcomes in the multi-dimensional space. A higher value of hypervolume indicates that the solutions are good in quality in terms of their proximity and diversity. However, in most cases, all outcomes within the maximum and minimum value may not be feasible. Therefore, it is not important to have a higher or lower value of this metric to be confident of convergence. In general cases, the optimization process is considered to be reasonably converged when the value of hyper-volume remains steady over a good number of function evaluations. 

ε-Progress simply defines the number of solutions that are to be included in the solution set for the next generations of the evolutionary algorithm. Similar to hyper-volume, stable progress over multiple function evaluations generally indicates a reasonable convergence. 

We may observe earlier stability in these metrics with a lower number of function evaluations. However, a higher number of function evaluations is recommended to track the convergence progress for a longer period of time, since it gives enough confidence that optimization is sufficiently converged and the obtained solutions are feasible and closer to optimality. Again, this number should conform to the requirement and sensitivity of the problem to avoid extensive computation time and cost associated with it. 

In [ ]:
# Import convergence metric classes and create an object

from emat.optimization import HyperVolume, EpsilonProgress, SolutionViewer, ConvergenceMetrics

convergence_metrics = ConvergenceMetrics(HyperVolume.from_outcomes(robustness_functions),
                                         EpsilonProgress(),
                                         SolutionViewer.from_model_and_outcomes(m, robustness_functions),)

#### Run robust optimization

As the model object, robustness measures, constraints and convergence metrics are all defined now, we can run the optimization process on the model object using `robust_optimize()` method. There are several inputs to this method:
* A list of robustness functions
* Number of scenarios to be generated for evaluating the performance of policy parameters
* Number of function evaluations
* A list of constraints
* Definition of Epsilon
* Convergence metrics, and
* Function evaluator 

As mentioned earlier, the number of function evaluations must be significantly higher in real applications to observe a good convergence that should return a set of optimal solutions we generally look for. `robust_optimize()` returns two `pandas` dataframes - one with the model parameters and robustness measures, and the other with convergence information. These dataframes are very handy in performing some post-analysis of the solutions across all function evaluations.  


In [ ]:
robust_results, convergence = m.robust_optimize(robustness_functions,
                                                scenarios = 500,
                                                nfe = 50000,
                                                constraints = [constraint_1, constraint_2],
                                                epsilons  =[0.05,]*len(robustness_functions),
                                                convergence = convergence_metrics,
                                                evaluator = eval_seq,)

In [ ]:
robust_results.head()

#### Plot convergence progress

The second dataframe returned by `robust_optimize()` contains the value of convergence metrics for each function evaluation and we can use this dataframe to visually inspect the convergence progress for the entire evaluation period. In this example, we considered 500 scenarios with 50,000 function evaluations. If we look at the ε-Progress plot, we see a stairs-like progress where the value becomes relatively stable after a certain leap. It is also evident that after 40,000 evaluations the value almost remains steady until the remain evaluation efforts are exhausted. For this use-case, this may be an indication of good convergence. However, there is no guarantee that we will not observe any better convergence for more function evaluations beyond this consideration. Getting a perfect convergence is always challenging in a multi-objective optimization problem, and in most cases, this is completely up to the decision analyst to decide on the trade-off between the evaluation time and convergence quality. 

Looking at the hyper-volume, we notice a few minor leaps that can be related to the rise in ε value in the left plot. The value of hyper-volume indicates a low ratio of dominated outcomes to all possible outcomes in the extent of the space. However, we observe steady progress after 30,000 evaluations which is a sign of reasonable convergence for this optimization setup. Like ε-progress, we might also observe more flatter portions indicating even better convergence for higher number of generations at the expense of higher evaluation time. 

In [ ]:
# Plot convergence metrics

fig, (ax1, ax2) = plt.subplots(ncols = 2, sharex = True, figsize=(15,4))

# Plot for Epsilon-progress
ax1.plot(convergence.nfe, convergence.epsilon_progress)
ax1.set_ylabel('$\epsilon$-progress')
ax1.set_xlabel('NFE')


# Plot for Hypervolume
ax2.plot(convergence.nfe, convergence.hypervolume)
ax2.set_ylabel('Hypervolume')
ax2.set_xlabel('NFE')
plt.show()

#### Plot parallel coordinate plot

We can also inspect the solutions obtained from robust optimization using a parallel coordinate plot.  This is an effective way of visualizing multi-variate information where each row in a dataframe is represented using a polyline and each point on a line indicates an attribute of that row. `emat.viz.parcoords` module helps create a parallel coordinate plot with the optimization results, so we can compare the policy levers side-by-side and examine their relationships with performance measures. The vertical axes represent the policy levers and performance measures used for robustness metrics. A polyline connecting different points on the vertical axes represents a solution.  We can specify a performance measure as the color dimension to better understand its relationship with other measures and policy parameters. 

In this example, we set 95th percentile value of unit cost for capacity expansion as color dimension where darker shade means higher value. We see that if we plan for a higher percentage of capacity expansion, we should expect a higher unit cost of expansion which is quite obvious. Again, if we expect less travel time and higher time savings due to capacity expansion, we must incur higher unit cost of expansion. We also observe some obvious relationships between measures such as travel time after expansion being inversely related to value of time savings. In some cases, a mix pattern is also noticed for certain policy levers such as expected net benefits or amortization period. If we go from left to right following a polyline in this plot, that gives us a solution combining different instances of policy levers that satisfy the constraints and are robust under uncertainty. In parallel coordinate plot, we have an assortment of all solutions of optimization that we can inspect visually to come up with the right set of decisions.  

In [ ]:
import emat.viz.parcoords

pc = emat.viz.parcoords.parallel_coords(
    m.ensure_dtypes(robust_results),
    model=m,
    robustness_functions=robustness_functions,
    color_dim = '95%ile Capacity Expansion Cost',
    colorscale = 'viridis',
    title = 'Robust Optimization Results : {} solutions'.format(robust_results.shape[0])
)

Show(pc, format = 'png', width = 1600, height = 400, scale = 2)

____